# CSE 151A Group Project: Tree Models

In [1]:
# All of our imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, poisson
import warnings

Run to suppress warnings when running code

In [2]:
warnings.filterwarnings('ignore')

First, we must get our processed data to train the model

In [3]:
input_df = pd.read_csv('processed_input.csv')
output_df = pd.read_csv('processed_output.csv')
display(input_df)
display(output_df)

,host_since,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bedrooms,price,minimum_nights,maximum_nights,...,Bathroom essentials,Baby safety gates,Bread maker,Clothing storage,Full kitchen,Fireplace guards,Sound system,EV charger,Outdoor furniture,Lake access
0,0.735956,0.0,0.000759,1.0,0.0,0.066667,0.0,0.002761,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.577049,0.0,0.000759,1.0,1.0,0.066667,0.0,0.006653,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.523716,0.0,0.000759,1.0,0.0,0.066667,0.0,0.004852,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.573115,0.0,0.000759,1.0,1.0,0.066667,0.0,0.003052,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.493989,0.0,0.000759,1.0,0.0,0.066667,0.0,0.003168,0.001002,0.013138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168318,0.467760,0.0,0.000759,1.0,1.0,0.066667,0.0,0.006653,0.000000,0.000070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168319,0.577486,0.0,0.000759,1.0,1.0,0.066667,0.0,0.003168,0.006012,0.000164,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168320,0.704044,0.0,0.000759,1.0,1.0,0.066667,0.0,0.002587,0.005010,0.000339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168321,0.447213,0.0,0.000759,1.0,1.0,0.066667,0.0,0.005782,0.002004,0.000199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,review_scores_rating
0,100.0
1,100.0
2,100.0
3,100.0
4,100.0
...,...
168318,100.0
168319,100.0
168320,100.0
168321,100.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(input_df,output_df, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=42)

print(f'Training data shapes: {X_train.shape}, {y_train.shape}')
print(f'Validation data shapes: {X_val.shape}, {y_val.shape}')
print(f'Test data shape: {X_test.shape}, {y_test.shape}')

Training data shapes: (107726, 247), (107726, 1)
Validation data shapes: (26932, 247), (26932, 1)
Test data shape: (33665, 247), (33665, 1)


In [5]:
validation_error_list = []
training_error_list = []
testing_error_list = []

In [6]:
decision_tree_model=DecisionTreeRegressor()
decision_tree_model.fit(X_train,y_train)

DecisionTreeRegressor()

In [7]:
# Show our starting off point for error
yhat_train = decision_tree_model.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
training_error_list.append(np.log(train_error))
print('Training Error:',train_error)

yhat_test = decision_tree_model.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
testing_error_list.append(np.log(test_error))
print('Testing Error:',test_error)

yhat_val = decision_tree_model.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
validation_error_list.append(np.log(val_error))
print('Validation Error:',val_error)

Training Error: 0.7772087016686742
Testing Error: 184.34098470186635
Validation Error: 180.97741277160424


In [8]:
# parameters={"splitter":["best","random"],
#             "max_depth" : [1,3,5,7,9,11,12],
#            "min_samples_leaf":[1,2,3,4,5,6,7,8,9,10],
#            "min_weight_fraction_leaf":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
#            "max_features":["auto","log2","sqrt",None],
#            "max_leaf_nodes":[None,10,20,30,40,50,60,70,80,90] }
# parameters = {
#     'splitter':["best","random"],
#     'max_depth': [3,5,7],
#     'min_samples_split': [3,5,7,9],
#     'min_samples_leaf': [3,5,7,9],
#     "max_leaf_nodes":[10,20,30,40]
# }
parameters = {'criterion':['friedman_mse','squared_error'],
              'max_depth':np.arange(1,15).tolist()[0::2],
              'min_samples_split':np.arange(2,11).tolist()[0::2],
              'max_leaf_nodes':np.arange(3,20).tolist()[0::2]}
tuning_model=GridSearchCV(decision_tree_model,parameters,scoring='neg_mean_squared_error',cv=10,verbose=0, n_jobs=1)

In [ ]:
tuning_model.fit(X_train,y_train)

In [ ]:
tuning_model.best_params_

In [ ]:
tuned_decision_tree_model = DecisionTreeRegressor(max_depth=7,max_leaf_nodes=19, min_samples_split=2, criterion='friedman_mse')
tuned_decision_tree_model.fit(X_train, y_train)

In [ ]:
# Show our starting off point for error
yhat_train = tuned_decision_tree_model.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
training_error_list.append(np.log(train_error))
print('Training Error:',train_error)

yhat_test = tuned_decision_tree_model.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
testing_error_list.append(np.log(test_error))
print('Testing Error:',test_error)

yhat_val = tuned_decision_tree_model.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
validation_error_list.append(np.log(val_error))
print('Validation Error:',val_error)

In [ ]:
#Random Forest Model
param_grid = {
    'n_estimators': [50, 100],  # Number of trees in the forest
    'max_depth': [10, 25],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 3]  # Minimum number of samples required to be at a leaf node
}
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
tuned_grid_search_model = RandomForestRegressor(max_depth=25, min_samples_leaf=1, min_samples_split=2, n_estimators=100)
tuned_grid_search_model.fit(X_train, y_train)

In [ ]:
#print train, test, and validation errors
yhat_train = tuned_grid_search_model.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
print('Training Error:',train_error)

yhat_test = tuned_grid_search_model.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
print('Testing Error:',test_error)

yhat_val = tuned_grid_search_model.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
print('Validation Error:',val_error)

In [11]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.5 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
# Gradient boosting models

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

model = xgb.XGBRegressor()

param_grid = {
    'learning_rate': [0.05, 0.1, 0.2],  # Learning rate
    'max_depth': [5,7,9],  # Maximum depth of a tree
    'n_estimators': [100, 200, 300],
    'subsample': [0.6, 0.7, 0.9],
    'colsample_bytree': [0.7, 0.9, 0.95],
    'gamma': [0, 0.1],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 2 folds for each of 32 candidates, totalling 64 fits
Best Parameters: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.9}
Best Score: 0.11548463891212407


In [39]:
xgbmodel = xgb.XGBRegressor(learning_rate= 0.1, max_depth= 7, n_estimators= 200, 
                         colsample_bytree= 0.7, subsample = 0.9, gamma=0)
xgbmodel.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [40]:
#Fitting 5 folds for each of 27 candidates, totalling 135 fits
#Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
#Best Score: 0.11855784874286188

In [41]:
yhat_train = xgbmodel.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
print('Training Error:',train_error)

yhat_test = xgbmodel.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
print('Testing Error:',test_error)

yhat_val = xgbmodel.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
print('Validation Error:',val_error)

Training Error: 68.12720297693896
Testing Error: 86.391349457696
Validation Error: 85.08594569433217


In [43]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5],
#     'subsample': [0.7, 0.8, 0.9],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['auto', 'sqrt', 'log2']
}

# Perform Grid Search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# View Results
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 2 folds for each of 27 candidates, totalling 54 fits
Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 300}
Best Score: 0.11223927176777004


In [45]:
gbmodel = GradientBoostingRegressor(learning_rate= 0.05, max_depth= 5, n_estimators= 300)
gbmodel.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=300)

In [46]:
yhat_train = gbmodel.predict(X_train)
train_error = mean_squared_error(y_train,yhat_train)
print('Training Error:',train_error)

yhat_test = gbmodel.predict(X_test)
test_error = mean_squared_error(y_test,yhat_test)
print('Testing Error:',test_error)

yhat_val = gbmodel.predict(X_val)
val_error = mean_squared_error(y_val,yhat_val)
print('Validation Error:',val_error)

Training Error: 80.27581729806296
Testing Error: 87.167911115311
Validation Error: 85.41331155949507
